In [6]:
import cv2
import numpy as np
import time
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import pickle
from keras.models import load_model

In [11]:
def extract_face_embedding(face_image, model):
    face_image = cv2.resize(face_image, (50, 50))
    face_image = face_image.astype('float32')

    mean, std = face_image.mean(), face_image.std()
    face_image = (face_image - mean) / std

    face_image = np.expand_dims(face_image, axis=0)

    embedding = model.predict(face_image)[0]

    return embedding

In [12]:
def load_train_data_and_extract_embeddings(train_data_dir, model):
    faces = []
    embeddings = []
    labels = []

    for person_name in os.listdir(train_data_dir):
        person_dir = os.path.join(train_data_dir, person_name)
        if not os.path.isdir(person_dir):
            continue
        
        for filename in os.listdir(person_dir):
            image_path = os.path.join(person_dir, filename)
            image = cv2.imread(image_path)
            face_embedding = extract_face_embedding(image, model)

            faces.append(image)
            embeddings.append(face_embedding)
            labels.append(person_name)
    
    return faces, embeddings, labels

In [13]:
# Đường dẫn tới thư mục chứa dữ liệu huấn luyện
train_data_dir = '../../../dataset/data/'

# Load pre-trained FaceNet model
faceNet_model = load_model('20240606-model.h5')

# Load training data and extract embeddings
faces, embeddings, labels = load_train_data_and_extract_embeddings(train_data_dir, faceNet_model)

# Chuyển đổi nhãn sang dạng số
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Huấn luyện một bộ phân loại SVM trên dữ liệu nhúng
svm_classifier = SVC(C=1.0, kernel='linear', probability=True)
svm_classifier.fit(embeddings, labels_encoded)

# Lưu trọng số của bộ phân loại và label encoder
with open('svm_classifier.pkl', 'wb') as f:
    pickle.dump(svm_classifier, f)
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv2d" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (1, 50, 50, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 50, 50, 3), dtype=float32)
  • training=False
  • mask=None

In [ ]:
# Hàm nhận diện khuôn mặt từ camera và dự đoán
def recognize_faces_from_camera(model, classifier, label_encoder):
    # Khởi tạo video stream từ camera
    vs = cv2.VideoCapture(0)
    time.sleep(2.0)

    while True:
        # Đọc frame từ video stream
        ret, frame = vs.read()
        if not ret:
            break

        # Resize frame để tăng tốc độ xử lý
        frame = cv2.resize(frame, (50, 50))

        # Chuyển đổi frame sang ảnh grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Sử dụng bộ phát hiện khuôn mặt Cascade Classifier của OpenCV
        face_cascade = cv2.CascadeClassifier('../utils/haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            # Phát hiện được khuôn mặt, cắt ảnh khuôn mặt và trích xuất embedding
            face_image = frame[y:y+h, x:x+w]
            face_embedding = extract_face_embedding(face_image, model)

            # Dự đoán nhãn của khuôn mặt
            prediction = classifier.predict([face_embedding])[0]
            probability = classifier.predict_proba([face_embedding])[0]
            name = label_encoder.inverse_transform([prediction])[0]
            prob = np.max(probability)

            # Vẽ hộp giới hạn và hiển thị kết quả dự đoán trên frame
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            text = f'{name}: {prob:.2f}'
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

        # Hiển thị frame
        cv2.imshow("Face Recognition", frame)

        # Thoát khỏi vòng lặp nếu nhấn phím 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Giải phóng video stream và đóng cửa sổ hiển thị
    vs.release()
    cv2.destroyAllWindows()

In [ ]:
# Load SVM classifier và label encoder từ file
with open('svm_classifier.pkl', 'rb') as f:
    loaded_classifier = pickle.load(f)
with open('label_encoder.pkl', 'rb') as f:
    loaded_label_encoder = pickle.load(f)

# Sử dụng mô hình nhận diện khuôn mặt và SVM classifier để nhận diện từ camera
recognize_faces_from_camera(faceNet_model, loaded_classifier, loaded_label_encoder)